In [1]:
#imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
from pyspark.sql.types import *

In [2]:
#instanciando spark 
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('desafio') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

23/06/07 03:17:16 WARN Utils: Your hostname, caio-HP-250-G8-Notebook-PC resolves to a loopback address: 127.0.1.1; using 192.168.1.122 instead (on interface wlo1)
23/06/07 03:17:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/07 03:17:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#diretorios de trabalho
cnaes_db = '/mnt/48447c52-52ac-4abb-be1b-d26a64732ee0/cursos/XP_Educação/engenheiro_de_dados_cloud/Modulo 2/base_dados/base_dados_desafio2/raw_data/csv/cnaes/cnaes.csv'
estabelecimentos_db = '/mnt/48447c52-52ac-4abb-be1b-d26a64732ee0/cursos/XP_Educação/engenheiro_de_dados_cloud/Modulo 2/base_dados/base_dados_desafio2/raw_data/csv/estabelecimentos/*'
cnaes_raw = '/mnt/48447c52-52ac-4abb-be1b-d26a64732ee0/cursos/XP_Educação/engenheiro_de_dados_cloud/Modulo 2/base_dados/base_dados_desafio2/raw_data/cnaes/'
estabelecimentos_raw = '/mnt/48447c52-52ac-4abb-be1b-d26a64732ee0/cursos/XP_Educação/engenheiro_de_dados_cloud/Modulo 2/base_dados/base_dados_desafio2/raw_data/estabelecimentos/'
stage_path = '/mnt/48447c52-52ac-4abb-be1b-d26a64732ee0/cursos/XP_Educação/engenheiro_de_dados_cloud/Modulo 2/base_dados/base_dados_desafio2/stage_data/'

In [6]:
##leitura da base de dados em formato csv

cnaes_df = (
    spark
    .read
    .format('csv')
    .option('header',True)
    .option('inferSchema',True)
    .option('delimiter',';')
    .option('encoding','ISO-8859-1')
    .load(cnaes_db)
    .select(
        col('CNAE').alias('id_cnae'),
        col('DESCRICAO_CNAE').alias('descricao_cnae')
    )
)

estab_df = (
    spark
    .read
    .format('csv')
    .option('header',True)
    .option('inferSchema',True)
    .option('delimiter',';')
    .option('encoding','ISO-8859-1')
    .load(estabelecimentos_db)
    .select(
        col('CNPJ_BASICO').alias('cnpj_basico'),
        col('CNPJ_ORDEM').alias('cnpj_ordem'),
        col('CNPJ_DV').alias('cnpj_dv'),
        col('IDENTIFICADOR_MATRIZ_FILIAL').alias('identificador_matriz_filial'),
        col('NOME_FANTASIA').alias('nome_fantasia'),
        col('SITUACAO_CADASTRAL').alias('situacao_cadastral'),
        col('DATA_SITUACAO_CADASTRAL').alias('data_situacao_cadastral'),
        col('MOTIVO_SITUACAO_CADASTRAL').alias('motivo_situacao_cadastral'),
        col('NOME_DA_CIDADE_NO_EXTERIOR').alias('nome_da_cidade_no_exterior'),
        col('PAIS').alias('pais'),
        col('DATA_INICIO_ATIVIDADE').alias('data_inicio_atividade'),
        col('CNAE_PRINCIPAL').alias('cnae_principal'),
        col('CNAE_SECUNDARIA').alias('cnae_secundaria'),
        col('TIPO_LOGRADOURO').alias('tipo_logradouro'),
        col('LOGRADOURO').alias('logradouro'),
        col('NUMERO').alias('numero'),
        col('COMPLEMENTO').alias('complemento'),
        col('BAIRRO').alias('bairro'),
        col('CEP').alias('cep'),
        col('UF').alias('uf'),
        col('MUNICIPIO').alias('municipio'),
        col('DDD_1').alias('ddd_1'),
        col('TEL_1').alias('tel_1'),
        col('DDD_2').alias('ddd_2'),
        col('TEL_2').alias('tel_2'),
        col('DDD_FAX').alias('ddd_fax'),
        col('FAX').alias('fax'),
        col('CORREIO_ELETRONICO').alias('correio_eletronico'),
        col('SITUACAO_ESPECIAL').alias('situacao_especial'),
        col('DATA_SITUACAO_ESPECIAL').alias('data_situacao_especial')
    )
)

23/06/07 01:04:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
#escrita dos arquivos em formato parquet

(
    cnaes_df
    .write
    .mode('overwrite')
    .format('parquet')
    .save(cnaes_raw)
)

(
    estab_df
    .write
    .mode('overwrite')
    .format('parquet')
    .save(estabelecimentos_raw)
)

In [4]:
##leitura da base de dados em formato parquet

cnaes_df = (
    spark
    .read
    .format('parquet')
    .option('header',True)
    .option('inferSchema',True)
    .option('encoding','ISO-8859-1')
    .load(cnaes_raw)
)

estab_df = (
    spark
    .read
    .format('parquet')
    .option('header',True)
    .option('inferSchema',True)
    .option('encoding','ISO-8859-1')
    .load(estabelecimentos_raw)
)

In [5]:
#Junção dos dataframes em um unico (JOIN)
joined_df = (
    estab_df
    .join(cnaes_df,estab_df.cnae_principal == cnaes_df.id_cnae,'inner')
    .select(
        'cnpj_basico',
        'cnpj_ordem',
        'cnpj_dv',
        'identificador_matriz_filial',
        'nome_fantasia',
        'situacao_cadastral',
        'data_situacao_cadastral',
        'motivo_situacao_cadastral',
        'nome_da_cidade_no_exterior',
        'pais',
        'data_inicio_atividade',
        'cnae_principal',
        col('descricao_cnae').alias('desc_cnae_principal'),
        'cnae_secundaria',
        'tipo_logradouro',
        'logradouro',
        'numero',
        'complemento',
        'bairro',
        'cep',
        'uf',
        'municipio',
        'ddd_1',
        'tel_1',
        'ddd_2',
        'tel_2',
        'ddd_fax',
        'fax',
        'correio_eletronico',
        'situacao_especial',
        'data_situacao_especial'   
    )
    
)

joined_df.show(truncate=False)

23/06/07 03:17:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+----------+-------+---------------------------+--------------------+------------------+-----------------------+-------------------------+--------------------------+----+---------------------+--------------+------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------+---------------+-------------------------------------------------+------+--------------------------+----------------------+--------+---+---------+-----+--------+-----+--------+-------+--------+------------------------------------+-----------------+----------------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|identificador_matriz_filial|nome_fantasia       |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_da_cidade_no_exterior|pais|data_inicio_atividade|cnae_principal|desc_cnae_principal                                                                                        

criando as tabelas sql

In [19]:

estab_df.createOrReplaceTempView('estabelecimentos')
joined_df.createOrReplaceTempView('joined')

Quantos estabelecimentos existem?

In [6]:
joined_df.count()

20996744

Na tabela de estabelecimentos, quantas colunas existem e quantas são
identificadas pelo spark como números? (Use inferSchema ao ler os arquivos).

resposta: 30 13

In [7]:
estab_df.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- cnpj_ordem: integer (nullable = true)
 |-- cnpj_dv: integer (nullable = true)
 |-- identificador_matriz_filial: integer (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: integer (nullable = true)
 |-- data_situacao_cadastral: integer (nullable = true)
 |-- motivo_situacao_cadastral: integer (nullable = true)
 |-- nome_da_cidade_no_exterior: string (nullable = true)
 |-- pais: integer (nullable = true)
 |-- data_inicio_atividade: integer (nullable = true)
 |-- cnae_principal: integer (nullable = true)
 |-- cnae_secundaria: string (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: integer (nullable = true)
 |-- ddd_1: string (nullable = true)

Vamos usar Spark SQL para obter algumas contagens. Primeiro, vamos ver
quantos estabelecimentos não tem logradouro cadastrado.
Use
estabelecimentos_df.createTempView("estabelecimentos")
para criar uma tabela temporária de logradouros e depois execute uma consulta
SQL como “SELECT COUNT(*) FROM estabelecimentos WHERE
LOGRADOURO IS NULL” para contar

In [21]:
(
    spark
    .sql('select count(*) from joined where logradouro is null')
    .show(truncate=False)
)

+--------+
|count(1)|
+--------+
|828     |
+--------+



Em muitos casos, as UDFs (funções definidas pelo usuário) são uma forma muito
conveniente de implementar uma lógica dentro de uma função python e chamá-la
dentro de uma consulta SQL. Vamos fazer isso para computar quantos
estabelecimento têm um logradouro cujo endereço é uma AVENIDA.
Crie uma função em Python “def is_avenida(logradouro):” que recebe um
logradouro e decide se ele é uma avenida ou não. Dica: Use startswith() e fique
atento ao resultado da questão anterior, pois ele pode impactar a lógica da sua
função. Esteja atento a maiúsculas e minúsculas.
Em seguida, use Spark SQL para executar uma consulta como "SELECT
COUNT(*) FROM estabelecimentos WHERE is_avenida(LOGRADOURO) == True” para contar quantos estabelecimentos ficam localizados em uma avenida.

In [9]:
#função que verifica se logradouro é avenida
def is_avenida(place):
    if(place.startswith('avenida') == True):
        return True
    return False
    
#registrando a função no spark
spark.udf.register('is_avenida',is_avenida)



#(lambda text: is_avenida(text))


<function __main__.is_avenida(place)>

In [22]:
(
    spark
    .sql('select count(*) as total_avenidas from \
               (select * from joined where logradouro is not null) as subQuery \
               where is_avenida(lower(logradouro)) == True')
    .show(truncate=False)

)

+--------------+
|total_avenidas|
+--------------+
|52587         |
+--------------+



Quantos CEPs distintos existem entre os estabelecimentos?

(no exercicio aparece o valor 889886 provavelmente esta errado)

In [23]:
(
    spark
    .sql('select count(distinct(cep)) from joined')
    .show()
)

+-------------------+
|count(DISTINCT cep)|
+-------------------+
|             889885|
+-------------------+



Quantos CNAEs existem na tabela de CNAES?

In [16]:
#checando se existem valores duplicados.

(
    cnaes_df
    .groupBy('id_cnae')
    .count()
    .sort(col('count').desc())
    .show(10)

)


+-------+-----+
|id_cnae|count|
+-------+-----+
| 113000|    1|
|1041400|    1|
|4322301|    1|
|7733100|    1|
|8730199|    1|
| 119907|    1|
|2013400|    1|
|2052500|    1|
|2449103|    1|
|3839499|    1|
+-------+-----+
only showing top 10 rows



In [17]:
cnaes_df.count()

1359

Vários CNAEs são de cultivo, como “cultivo de milho”, “cultivo de trigo” e “cultivo
de arroz” Quantos estabelecimentos possuem um CNAE relacionado a cultivo?
Dicas: use a operação JOIN para criar uma dataframe que tem os
estabelecimentos e as descrições de seus CNAES. Crie uma UDF que verifica se
a descrição do CNAE é sobre cultivo; e usar Spark SQL para escrever uma
consulta como “SELECT COUNT(*) from estabelecimentos_with_cnae WHERE
is_cnae_cultivo(DESCRICAO_CNAE) == True”.

In [18]:
#função que verifica se cnae é relacionado a cultivo
def is_cultivo(word):
    if(word.startswith('cultivo') == True):
        return True
    return False
    
#registrando a função no spark
spark.udf.register('is_cultivo',is_cultivo)


<function __main__.is_cultivo(word)>

In [25]:
(
    spark
    .sql('select count(*) as total_cultivo from joined where is_cultivo(lower(desc_cnae_principal)) == True')
    .show(truncate=False)

)

+-------------+
|total_cultivo|
+-------------+
|200243       |
+-------------+



Quantos estabelecimentos são filiais? Consulte o dicionário dos dados e use
Spark SQL.

In [28]:
(
    spark
    .sql('select count(*) as count_filiais from joined where identificador_matriz_filial == 2 ')
    .show(truncate=False)
)

+-------------+
|count_filiais|
+-------------+
|1093082      |
+-------------+

